In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [2]:
raw_df = pd.read_csv('data/288821rows.csv')

C:\Users\Sean\AppData\Local\Temp\ipykernel_10960\627538862.py:1: DtypeWarning: Columns (10,11,63,66,68,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('288821rows.csv')


In [3]:
df = raw_df[['num_comments', 'title']]

In [4]:
df.loc[:,'title'] = df['title'].str.lower()

C:\Users\Sean\AppData\Local\Temp\ipykernel_10960\989622944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'title'] = df['title'].str.lower()


In [5]:
tokenizer = nltk.RegexpTokenizer(r'\b[(a-z)]+\b')

df.loc[:,'tokenized'] = df['title'].apply(tokenizer.tokenize)

C:\Users\Sean\AppData\Local\Temp\ipykernel_10960\1085512727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'tokenized'] = df['title'].apply(tokenizer.tokenize)


In [6]:
stop_words = nltk.corpus.stopwords.words('english')

def rm_stop(words):
    new_words = []
    for w in words:
        if w not in stop_words:
            new_words.append(w)
    return new_words

In [7]:
df['tokenized'] = df['tokenized'].apply(rm_stop)
df['text'] = df['tokenized'].apply(lambda x: ' '.join([word for word in x]))

C:\Users\Sean\AppData\Local\Temp\ipykernel_10960\2842049158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokenized'] = df['tokenized'].apply(rm_stop)
C:\Users\Sean\AppData\Local\Temp\ipykernel_10960\2842049158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['tokenized'].apply(lambda x: ' '.join([word for word in x]))


In [8]:
df

,num_comments,title,tokenized,text
0,4,what's the best thing to say to someone who's ...,"[best, thing, say, someone, upset]",best thing say someone upset
1,8,where would someone be able to get their hands...,"[would, someone, able, get, hands, coke]",would someone able get hands coke
2,4,what’s your ideal first date?,"[ideal, first, date]",ideal first date
3,3,"law enforcement agents of reddit, what do the ...","[law, enforcement, agents, reddit, shows, movi...",law enforcement agents reddit shows movies get...
4,3,what sounds innocent but also condescending?,"[sounds, innocent, also, condescending]",sounds innocent also condescending
...,...,...,...,...
288816,8,"those who decided not to get married, why?","[decided, get, married]",decided get married
288817,9,"if one body part could be easily replaceable, ...","[one, body, part, could, easily, replaceable, ...",one body part could easily replaceable one wou...
288818,29,those who don’t have/like to use a bidet at ho...,"[like, use, bidet, home]",like use bidet home
288819,12,how did you find out about / get on reddit?,"[find, get, reddit]",find get reddit


In [17]:
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['text'])
bow = X.toarray()
vocab = vectorizer.get_feature_names()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)